## Cheatsheet - download from SFTP with pysftp - Financial purpose

How download csv file from SFTP. Here the code with pysftp.

The purpose in my case is to acumulate all finalncial transactions from the payment gateway

In [ ]:
# in case the library is not installed

import pysftp

### Formatting the file name

In [ ]:
# In my case the needed is download all transactions from the day before the current one
# I'll use datetime to reach the purpose

from datetime import datetime, timedelta

date = datetime.today() - timedelta(days=1)
year = date.year
month = date.month
day = date.day

file_name = "PATTERN-FILE-NAME_{}_{:02d}_{:02d}.csv".format(year, month, day) # 2 digits on month and day ## for more information visit https://pyformat.info/

print (file_name)

### Connection to one sftp - result: one file .csv

In [ ]:
myHostname = "YOURSERVERDOMAINORIP.COM"
myUsername = "ROOT"
myPassword = "12345"

In [ ]:
# First way

cnopts = pysftp.CnOpts()
cnopts.hostkeys = None

with pysftp.Connection(host=myHostname, username=myUsername, password=myPassword, cnopts=cnopts) as sftp:
    print ("Connection succesfully stablished ... ")
    remoteFilePath = '/FTP-DIRECTORY/'+file_name
    localFilePath = '/LOCAL-FILE-DIRECTORY/'+file_name
    sftp.get(remoteFilePath, localFilePath, callback=None) 

In [ ]:
# Second way

# Defining the function to create the connection  

def create_connection(host, port, username, password):
        cnopts = pysftp.CnOpts()
        cnopts.hostkeys = None

        try:
            conn = pysftp.Connection(host,
                                     port=port,
                                     username=username,
                                     password=password,
                                     cnopts=cnopts)
            return conn

        except Exception as error:
            print("ERROR: impossible to create connection")


# Defining the function to download the .csv to the path indicated (using create_connection function)

def download_file(conn, remoteFilePath, localFilePath):
    conn.get(remoteFilePath, localFilePath, callback=None)


# Iterating all accounts and creating the final accumulated one

conn = create_connection(account["myHostname"], account["myPort"], account["myUsername"], account["myPassword"])
print ("Connection succesfully stablished ... ")
download_file(conn, '/FTP-DIRECTORY/'+file_name, '/LOCAL-FILE-DIRECTORY/'+file_name)
conn.close()

#### Connection to multiple sftp - result: one accumulated file .csv

In [ ]:
# Using list of dictionaries to iterate on each account

accounts = [
    {
      "myAccount": "ACCOUNT-NAME-1",
      "myHostname": "YOURSERVERDOMAINORIP.COM-1",
      "myUsername": "ROOT-1",
      "myPassword": "12345-1",
      "myPort": 22
    },
    {
      "myAccount": "ACCOUNT-NAME-2",
      "myHostname": "YOURSERVERDOMAINORIP.COM-2",
      "myUsername": "ROOT-2",
      "myPassword": "12345-2",
      "myPort": 22
    }
]

In [ ]:
# Pandas will help to iterate on each account and concat the data to a final df

import pandas as pd

In [ ]:
# Defining the function to create the connection  

def create_connection(host, port, username, password):
        cnopts = pysftp.CnOpts()
        cnopts.hostkeys = None

        try:
            conn = pysftp.Connection(host,
                                     port=port,
                                     username=username,
                                     password=password,
                                     cnopts=cnopts)
            return conn

        except Exception as error:
            print("ERROR: impossible to create connection")


# Defining the function to download the .csv to the path indicated (using create_connection function)

def download_file(conn, remoteFilePath, localFilePath):
    conn.get(remoteFilePath, localFilePath, callback=None)


# Iterating all accounts and creating the final accumulated one

columns = ["NAMES","OF","COLUMNS","IN","YOUR","CSV"] # In case is needed standardize the names of the columns
concatenated_df = pd.DataFrame(columns=columns)

for account in accounts:
    print("downloading " + account["myAccount"])
    conn = create_connection(account["myHostname"], account["myPort"], account["myUsername"], account["myPassword"])
    download_file(conn, '/FTP-DIRECTORY/'+file_name, '/LOCAL-FILE-DIRECTORY/'+file_name)
    conn.close()

    df = pd.read_csv('/LOCAL-FILE-DIRECTORY/'+file_name) 
    
    print(df.shape)
    
    if len(df.index) < 1: # excluding empty df
        print("empty file " + file_name)
        continue
        
    concatenated_df = pd.concat([concatenated_df, df])

concatenated_df.to_csv('/LOCAL-FILE-DIRECTORY/' + file_name) # convert the df to .csv file
